In [11]:
import pandas as pd
import xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn import model_selection
import scipy 
from sklearn.model_selection import GridSearchCV   #Perforing grid search


train_data = pd.read_csv('df_training.csv')
test_data = pd.read_csv('df_test.csv')

target = 'TARGET'
IDcol = 'ID'

predictors = [x for x in train_data.columns if x not in [target,IDcol]]

#train_data = train_data.dropna()
#train_data = train_data.fillna(train_data.median())
#test_data = test_data.fillna(test_data.median())
#train_data.apply()
#train_data[['TLSum']]#.apply(scipy.stats.kurtosis)     
#train_data.TLSum = (train_data[['TLSum']]+50).apply(np.log)
#train_data.TLMaxSum = (train_data[['TLMaxSum']]+100).apply(np.log)
#train_data[['TLMaxSum']] .apply(scipy.stats.kurtosis)   

#a.apply(scipy.stats.kurtosis)         
#a.apply(scipy.stats.kurtosis)  

In [28]:
param_test0 = {
 'n_estimators':range(36,40,1),
 'learning_rate':[0.45,0.5,0.55]
}
param_test1 = {
 'max_depth':range(6,10,1),
 'min_child_weight':range(5,8,1)
}
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
param_test5 = {
 'subsample':[i/100.0 for i in range(54,66,2)],
 'colsample_bytree':[i/100.0 for i in range(70,80,2)]
}

param_test6 = {
 'reg_alpha':[0.80,0.85,0.75]
}
param_test7 = {
 'scale_pos_weight':[4.5,5,5.5]
}
gsearch0 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.5, n_estimators=37, max_depth=7,
 min_child_weight=6, gamma=0, subsample=0.60, colsample_bytree=0.76,reg_alpha=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=5,seed=27), 
 param_grid = param_test0, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch0.fit(train_data[predictors],train_data[target])
gsearch0.best_params_,    gsearch0.best_score_


({'learning_rate': 0.5, 'n_estimators': 37}, 0.3960913117889862)

In [3]:
train_data.head()

,ID,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,TLCnt03,...,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct,TARGET
0,478,0,0,0,1,4.0,1,487,12,0,...,0.7681,0.8182,0,0,0,0.5909,0,0,0.4615,0
1,87963,0,0,0,4,1.0,3,283,6,0,...,0.7108,0.7273,0,0,1,0.5909,0,0,0.1538,0
2,35365,2,1,0,2,2.0,5,252,11,0,...,0.4656,0.0526,3,7,14,0.5263,9,9,0.1000,1
3,8250,0,0,0,1,3.0,6,165,27,0,...,0.8179,0.0714,2,9,11,0.5714,9,10,0.0000,1
4,86359,0,0,0,3,1.0,3,331,6,0,...,0.6032,0.6500,3,2,3,0.7500,2,3,0.4000,0


In [239]:


# 把数据集拆分成训练集和测试集
seed = 7
test_size = 0.2
X = train_data.iloc[:,1:-1]
y = train_data.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=test_size,random_state=1)

# 训练样本归一化
std_x = MinMaxScaler()
x_train = std_x.fit_transform(x_train)
x_test = std_x.transform(x_test)

# 测试样本归一化
std_y  = MinMaxScaler()
y_train = std_y.fit_transform(y_train.values.reshape(-1,1))
y_test = std_y.transform(y_test.values.reshape(-1,1))


# 所有训练样本的归一化
std_x_all = MinMaxScaler()
x_train_all = std_x_all.fit_transform(X)

std_y_all = MinMaxScaler()
y_train_all = std_y_all.fit_transform(y.values.reshape(-1,1))

# 预测值处理
std_x_test = MinMaxScaler()
x_test_data = std_x_test.fit_transform(test_data.iloc[:,1:-1])


def f1_score_vail(pred, data_vail):
    labels = data_vail.get_label() 
    preds = []
    for i in pred: 
        if i > 0.5:
            preds.append(1)
        else:
            preds.append(0) 
    score_vail = f1_score(y_true=labels, y_pred=preds, average='binary')      # xgb的predict输出即为对应的label
    return '1-f1_score', 1-score_vail   # xgb目标是将目标指标降低


In [240]:

param_test1 = {
 'max_depth':range(3,10,1),
 'learning_rate':np.arange(0.01,0.2,0.01)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(scale_pos_weight=5,feval=f1_score_vail
                            ), 
                         param_grid = param_test1,
                         scoring='f1',
                         n_jobs=4,
                         iid=False,
                         cv=5
                       )
gsearch1.fit(x_train,y_train)

# 对测试集做预测
y_pred = gsearch1.predict(x_test)
predictions = [round(value) for value in y_pred]
 
# 评估预测结果
precision_score = metrics.precision_score(y_test, predictions)
print("precision_score: %.2f%%" % (precision_score * 100.0)) 

# 评估预测结果
recall = recall_score(y_test, predictions)
print("recall: %.2f%%" % (recall * 100.0)) 


# 评估预测结果
confusions = metrics.confusion_matrix(y_test, predictions)
print(confusions) 

# 评估预测结果
f1 = f1_score(y_test, predictions)
print("f1: %.2f%%" % (f1 * 100.0))
#gsearch1.best_params_,     gsearch1.best_score_

precision_score: 39.34%
recall: 67.61%
[[181  74]
 [ 23  48]]
f1: 49.74%


In [30]:
test_data[predictors].head()

,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,TLCnt03,TLCnt12,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,0,1,1,1.0,0,106,31,0,0,...,0.0,0.0000,0.1429,0,1,4,0.1429,1,1,0.0000
1,0,0,0,3,1.0,6,227,6,0,6,...,10.0,0.7670,0.5862,3,0,1,0.4828,0,1,0.8571
2,5,4,0,2,1.0,4,117,7,0,2,...,2.0,0.9365,0.3000,1,2,5,0.2000,6,2,1.2500
3,0,0,0,0,NaN,0,334,22,0,0,...,3.0,0.6650,0.7500,0,0,0,0.6667,0,0,0.1250
4,0,0,0,1,6.0,0,275,3,1,1,...,0.0,0.0477,0.6400,0,0,0,0.5200,0,0,0.3846


In [32]:

#保存预测结果
#y_predict = RF.predict(x_test_data)
y_predict = gsearch0.predict(test_data[predictors])
df = pd.DataFrame(columns=['ID','Predicted_Results'])
df['ID'] = test_data.iloc[:,0]
df['Predicted_Results'] = y_predict
df.to_csv('output/result0929.csv',index=False)


In [34]:
df.Predicted_Results.value_counts()

0    493
1    107
Name: Predicted_Results, dtype: int64

In [35]:
df545= pd.read_csv('output/result0.545.csv')

In [36]:
df545.Predicted_Results.value_counts()

0    447
1    153
Name: Predicted_Results, dtype: int64

In [38]:
train_data.TARGET.value_counts()

0    1500
1     300
Name: TARGET, dtype: int64